# Lecture 1 - Handling Missing Values
Loại bỏ giá trị thiếu, hoặc điền chúng vào bằng một quy trình tự động.

## Xem qua dữ liệu lần đầu

Điều đầu tiên chúng ta cần làm là tải các thư viện và bộ dữ liệu mà chúng ta sẽ sử dụng.

Để minh họa, chúng ta sẽ sử dụng một bộ dữ liệu về các sự kiện đã xảy ra trong các trận đấu Bóng bầu dục Mỹ. Trong [**bài tập tiếp theo**](https://www.kaggle.com/kernels/fork/10824396), bạn sẽ áp dụng các kỹ năng mới của mình vào một bộ dữ liệu về giấy phép xây dựng đã được cấp ở San Francisco.

In [1]:
# Các module chúng ta sẽ sử dụng
import pandas as pd
import numpy as np

# Đọc toàn bộ dữ liệu của chúng ta
nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")

# Đặt seed để đảm bảo tính tái tạo
np.random.seed(0)

C:\Users\minh PC\AppData\Local\Temp\ipykernel_4708\2378803584.py:6: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")


Điều đầu tiên cần làm khi bạn nhận được một bộ dữ liệu mới là xem qua một phần của nó. Điều này giúp bạn xác nhận rằng dữ liệu đã được đọc đúng và cung cấp một cái nhìn tổng quan về dữ liệu. Trong trường hợp này, hãy kiểm tra xem có giá trị thiếu nào không, những giá trị này sẽ được biểu diễn bằng `NaN` hoặc `None`.

In [2]:
# Xem năm dòng đầu tiên của tệp nfl_data.
# Tôi có thể thấy một vài dữ liệu thiếu ngay lập tức!
nfl_data.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Đúng vậy, có vẻ như có một số giá trị thiếu.

## Chúng ta có bao nhiêu điểm dữ liệu thiếu?

Ok, bây giờ chúng ta đã biết là có một số giá trị thiếu. Hãy cùng xem chúng ta có bao nhiêu giá trị thiếu trong mỗi cột.


In [3]:
# Lấy số lượng điểm dữ liệu thiếu theo từng cột
missing_values_count = nfl_data.isnull().sum()

# Xem số lượng điểm dữ liệu thiếu trong mười cột đầu tiên
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

Có vẻ như đó là một con số khá lớn! Sẽ rất hữu ích nếu chúng ta xem xét tỷ lệ phần trăm các giá trị bị thiếu trong bộ dữ liệu của mình để có cái nhìn rõ hơn về quy mô của vấn đề này:


In [4]:
# Tổng số giá trị thiếu là bao nhiêu?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# Tỷ lệ phần trăm dữ liệu bị thiếu
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

27.66722370547874


Wow, gần một phần tư số ô trong bộ dữ liệu này là trống! Ở bước tiếp theo, chúng ta sẽ xem xét kỹ hơn một số cột có giá trị thiếu và cố gắng tìm hiểu xem có vấn đề gì với chúng.

## Tìm hiểu lý do dữ liệu bị thiếu

Đây là bước mà chúng ta sẽ đi vào phần khoa học dữ liệu mà tôi thích gọi là "trực giác dữ liệu", có nghĩa là "thực sự nhìn vào dữ liệu của bạn và cố gắng tìm hiểu lý do tại sao nó lại như vậy và điều đó sẽ ảnh hưởng thế nào đến phân tích của bạn". Đây có thể là một phần khá gây bực bội trong khoa học dữ liệu, đặc biệt là nếu bạn mới gia nhập lĩnh vực này và chưa có nhiều kinh nghiệm. Để xử lý các giá trị thiếu, bạn sẽ cần phải sử dụng trực giác của mình để tìm hiểu lý do tại sao giá trị đó bị thiếu. Một trong những câu hỏi quan trọng nhất bạn có thể tự hỏi để giúp tìm ra điều này là:

> **Liệu giá trị này bị thiếu vì nó không được ghi lại hay vì nó không tồn tại?**

Nếu giá trị bị thiếu vì nó không tồn tại (như chiều cao của đứa trẻ lớn nhất của một người không có con) thì sẽ không hợp lý khi cố gắng đoán giá trị đó có thể là gì. Những giá trị này bạn có thể muốn giữ lại là `NaN`. Ngược lại, nếu giá trị bị thiếu vì nó chưa được ghi lại, thì bạn có thể cố gắng đoán nó có thể là gì dựa trên các giá trị khác trong cột và hàng đó. Đây được gọi là **làm đầy dữ liệu (imputation)**, và chúng ta sẽ học cách làm điều đó trong bước tiếp theo! :)

Hãy cùng làm một ví dụ. Nhìn vào số lượng giá trị thiếu trong dataframe `nfl_data`, tôi nhận thấy rằng cột "TimesSec" có rất nhiều giá trị thiếu:


In [5]:
# Xem số lượng điểm dữ liệu thiếu trong mười cột đầu tiên
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

Bằng cách xem qua [tài liệu](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), tôi có thể thấy rằng cột này chứa thông tin về số giây còn lại trong trận đấu khi pha chơi được thực hiện. Điều này có nghĩa là các giá trị này có thể bị thiếu vì chúng không được ghi lại, thay vì vì chúng không tồn tại. Vì vậy, sẽ hợp lý hơn nếu chúng ta cố gắng đoán giá trị của chúng thay vì chỉ để chúng là NA.

Mặt khác, có những trường khác, như "PenalizedTeam", cũng có rất nhiều giá trị thiếu. Tuy nhiên, trong trường hợp này, trường này bị thiếu vì nếu không có phạt, thì không hợp lý để nói *đội nào* bị phạt. Đối với cột này, sẽ hợp lý hơn nếu để trống hoặc thêm một giá trị thứ ba như "neither" và sử dụng nó để thay thế các giá trị NA.

> **Mẹo:** Đây là một nơi tuyệt vời để đọc tài liệu bộ dữ liệu nếu bạn chưa làm vậy! Nếu bạn đang làm việc với bộ dữ liệu được lấy từ người khác, bạn cũng có thể thử liên hệ với họ để có thêm thông tin.

Nếu bạn đang làm phân tích dữ liệu rất cẩn thận, đây là lúc bạn sẽ xem xét từng cột một để tìm ra chiến lược tốt nhất để điền các giá trị thiếu. Trong phần còn lại của notebook này, chúng ta sẽ đề cập đến một số kỹ thuật "nhanh và bẩn" có thể giúp bạn với các giá trị thiếu, nhưng có thể cũng sẽ loại bỏ một số thông tin hữu ích hoặc thêm một số nhiễu vào dữ liệu của bạn.

## Loại bỏ giá trị thiếu

Nếu bạn đang vội hoặc không có lý do để tìm hiểu tại sao giá trị của bạn bị thiếu, một lựa chọn bạn có là chỉ cần loại bỏ bất kỳ hàng hoặc cột nào chứa giá trị thiếu. (Lưu ý: Tôi không thường xuyên khuyến khích phương pháp này cho các dự án quan trọng! Thường thì việc dành thời gian để xem xét dữ liệu của bạn và thực sự nhìn vào tất cả các cột có giá trị thiếu một cách kỹ lưỡng sẽ đáng giá hơn.)

Nếu bạn chắc chắn muốn loại bỏ các hàng có giá trị thiếu, pandas có một hàm tiện dụng, `dropna()`, để giúp bạn làm điều này. Hãy thử nó với bộ dữ liệu NFL của chúng ta!


In [6]:
# Loại bỏ tất cả các hàng chứa giá trị thiếu
nfl_data.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Ôi không, có vẻ như điều đó đã xóa hết toàn bộ dữ liệu của chúng ta! 😱 Điều này xảy ra vì mỗi hàng trong bộ dữ liệu của chúng ta đều có ít nhất một giá trị thiếu. Có thể chúng ta sẽ gặp may hơn nếu loại bỏ tất cả các *cột* có ít nhất một giá trị thiếu thay vì loại bỏ các hàng.


In [7]:
# Loại bỏ tất cả các cột có ít nhất một giá trị thiếu
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009


In [8]:
# Chúng ta đã mất bao nhiêu dữ liệu?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's dropped: 37


Chúng ta đã mất khá nhiều dữ liệu, nhưng ở thời điểm này, chúng ta đã thành công trong việc loại bỏ tất cả các giá trị `NaN` khỏi dữ liệu của mình.

## Điền giá trị thiếu tự động

Một lựa chọn khác là cố gắng điền các giá trị thiếu. Đối với phần tiếp theo, tôi sẽ lấy một phần nhỏ của dữ liệu NFL để nó có thể in ra dễ dàng hơn.


In [9]:
# Lấy một phần nhỏ của bộ dữ liệu NFL
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Chúng ta có thể sử dụng hàm `fillna()` của Panda để điền giá trị thiếu trong một dataframe. Một lựa chọn chúng ta có là chỉ định giá trị mà chúng ta muốn thay thế các giá trị `NaN`. Ở đây, tôi đang nói rằng tôi muốn thay thế tất cả các giá trị `NaN` bằng 0.


In [10]:
# Thay thế tất cả các giá trị NA bằng 0
subset_nfl_data.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


Tôi cũng có thể thông minh hơn một chút và thay thế các giá trị thiếu bằng bất kỳ giá trị nào ngay sau đó trong cùng một cột. (Điều này rất hợp lý đối với các bộ dữ liệu mà các quan sát có một thứ tự logic nhất định.)


In [11]:
# Thay thế tất cả các giá trị NA bằng giá trị ngay sau đó trong cùng một cột,
# sau đó thay thế tất cả các giá trị NA còn lại bằng 0
subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)

C:\Users\minh PC\AppData\Local\Temp\ipykernel_4708\791205422.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)


,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


## Your turn

Write your own code to [**deal with missing values**](https://www.kaggle.com/kernels/fork/10824396) in a dataset of building permits issued in San Francisco.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/data-cleaning/discussion) to chat with other learners.*